<a href="https://colab.research.google.com/github/shashank1947/NNDL/blob/main/NNDl_assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error

In [8]:
data = pd.read_excel('/content/drive/MyDrive/Load Data in kW.xlsx')
data.head()

,Date,Time,Load (kW)
0,01.09.2018,00:00:00,5551.82208
1,NaN,01:00:00,4983.17184
2,NaN,02:00:00,4888.39680
3,NaN,03:00:00,5072.95872
4,NaN,04:00:00,5196.25980


In [9]:
X = []
Y = []
for i in range(24, data.shape[0]):
    X.append(data['Load (kW)'].iloc[i-24*7]) # previous week
    Y.append(data['Load (kW)'].iloc[i])

In [10]:
data = pd.DataFrame({'Prev Day Load' : X, 'Pres Day Load' : Y})
data.head()

,Prev Day Load,Pres Day Load
0,5484.94956,4931.26380
1,5115.35808,4775.53968
2,4853.47968,4713.81120
3,4696.35264,4689.02628
4,4256.45928,4844.75040


In [11]:
Xmax = np.max(data['Prev Day Load'])
Xmin = np.min(data['Prev Day Load'])
Ymax = np.max(data['Pres Day Load'])
Ymin = np.min(data['Pres Day Load'])
data['Prev Day Load'] = (data['Prev Day Load'] - Xmin) / (Xmax - Xmin)
data['Pres Day Load'] = (data['Pres Day Load'] - Ymin) / (Ymax - Ymin)

In [12]:
x = data['Prev Day Load']
y = data['Pres Day Load']
Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.1, random_state=101)

In [13]:
#Initialization
m = np.random.uniform(-2, 2)
c = np.random.uniform(-2, 2)
eta = 0.1
epochs = 500
gm2 = 0
gc2 = 0
e = 1E-6

# Loop for epochs
for _ in range(epochs):
    
    for i in range(Xtrain.shape[0]):
        # Calculating Grad M
        gradM = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c) * Xtrain.iloc[i]
    
        # Calculating Grad C
        gradC = -1 * (Ytrain.iloc[i] - m * Xtrain.iloc[i] - c)
        
        # Calculating updated values of gm2 and gc2
        gm2 += (gradM ** 2)
        gc2 += (gradC ** 2)
        
        # Updating m and c values
        m -= (eta * gradM) / ((e + gm2) ** 0.5)
        c -= (eta * gradC) / ((e + gc2) ** 0.5)

# Print model parameters after training
print(f'm = {m}\nc = {c}')

m = 0.4474994969520615
c = 0.26303483620529433


In [14]:
train_pred = [m * Xtrain.iloc[i] + c for i in range(Xtrain.shape[0])]
test_pred = [m * Xtest.iloc[i] + c for i in range(Xtest.shape[0])]

In [15]:
train_pred = [i * (Ymax - Ymin) + Ymin for i in train_pred]
test_pred = [i * (Ymax - Ymin) + Ymin for i in test_pred]
trainY = [i * (Ymax - Ymin) + Ymin for i in Ytrain]
testY = [i * (Ymax - Ymin) + Ymin for i in Ytest]

In [16]:
# Training Error Calculation
training_MAE = sum([abs(train_pred[i] - trainY[i]) for i in range(len(trainY))]) / len(trainY)
training_MSE = sum([(train_pred[i] - trainY[i]) ** 2 for i in range(len(trainY))]) / len(trainY)
training_RMSE = training_MSE ** 0.5
print('Training Error :')
print(f'MAE : {training_MAE}')
print(f'MSE : {training_MSE}')
print(f'RMSE : {training_RMSE}')

# Testing Error Calculation
testing_MAE = sum([abs(test_pred[i] - testY[i]) for i in range(len(testY))]) / len(testY)
testing_MSE = sum([(test_pred[i] - testY[i]) ** 2 for i in range(len(testY))]) / len(testY)
testing_RMSE = testing_MSE ** 0.5
print('Testing Error :')
print(f'MAE : {testing_MAE}')
print(f'MSE : {testing_MSE}')
print(f'RMSE : {testing_RMSE}')

Training Error :
MAE : 748.7814268906316
MSE : 908589.9487767653
RMSE : 953.1998472391638
Testing Error :
MAE : 794.6932671941701
MSE : 1048420.8357584595
RMSE : 1023.9242334071694


In [17]:
pd.DataFrame({'Actual Values' : trainY, 'Predicted Values' : train_pred})

,Actual Values,Predicted Values
0,4221.85392,5819.147208
1,7609.43808,6763.018643
2,5847.05880,4998.814042
3,4672.50300,6154.744390
4,7883.78688,6542.519226
...,...,...
1939,5830.37964,5741.020240
1940,6110.49600,5533.425725
1941,5289.94368,5661.916685
1942,7962.97392,6709.794645


In [18]:
pd.DataFrame({'Actual Values' : testY, 'Predicted Values' : test_pred})

,Actual Values,Predicted Values
0,6161.624640,6081.151577
1,5761.792440,6779.411355
2,7390.270800,6234.615264
3,4597.368840,5927.687889
4,5242.712040,5444.277273
...,...,...
211,6764.319072,6595.743223
212,7390.270800,7001.933701
213,6550.077600,6234.615264
214,5722.043040,5825.843806


In [19]:
x = float(input('Enter the load at present hour on previouus day : '))
x = (x - Xmin) / (Xmax - Xmin)
prediction = m * x + c
prediction = (prediction * (Ymax - Ymin)) + Ymin
print('Predicted load at present hour :', prediction)

Enter the load at present hour on previouus day : 3865
Predicted load at present hour : 5033.044388727706
